<a href="https://colab.research.google.com/github/sarang-manohar/Coursera_Capstone/blob/main/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook puts in the seed of the Capstone Project with Coursera!

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
import folium
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim

print('Libraries imported')

Libraries imported


In [3]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(html_data.text, 'html.parser')
table = soup.find("table",{"rules":"all"})

table_data = []

for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_data.append(cell)

df=pd.DataFrame(table_data)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df['Address'] = df['Borough']+', Toronto, Ontario, Canada, '+df['PostalCode']
df.head()

,PostalCode,Borough,Neighborhood,Address
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A"
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A"
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A"


In [4]:
lat=[]
long=[]
for i, pc in enumerate(df['PostalCode']):
    with requests.Session() as session:
        g = geocoder.arcgis('{}, Toronto, Canada'.format(pc),session=session)
    lat.append(g.latlng[0])
    long.append(g.latlng[1])

df['Latitude'] = lat
df['Longitude'] = long

df.head()

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A",43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A",43.66253,-79.39188


In [5]:
address = 'Toronto, Ontario, Canada'

g = geocoder.arcgis(address)
latitude = g.latlng[0]
longitude = g.latlng[1]
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.648690000000045, -79.38543999999996.


In [6]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto) 

map_Toronto

In [7]:
CLIENT_ID = 'Q5LJU1V0DPCB0R5XDREYLILRZGTSPSBFPBX1QGAQHZAAPNPK' # your Foursquare ID
CLIENT_SECRET = 'U4XJXABOBZBSHOWMLEQYHNEYVSXAR0VBAYAUGHLBBQ3LJIVX' # your Foursquare Secret
VERSION = '20210704' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = '0HUXGDI5F3EDNXPW3RW21KJNV4TB3Z4ITMA34M5VRKXR0HRA'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q5LJU1V0DPCB0R5XDREYLILRZGTSPSBFPBX1QGAQHZAAPNPK
CLIENT_SECRET:U4XJXABOBZBSHOWMLEQYHNEYVSXAR0VBAYAUGHLBBQ3LJIVX


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&GET'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION)
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.drop({'categories'},axis=1,inplace=True)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

<ipython-input-9-54553548249f>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


In [10]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [11]:
df.head(1)

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991


In [12]:
df_test = df.copy()
df_test = df_test.head(2)
df_test

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A",43.73057,-79.31306


In [13]:
venues_list=[]
for name, lat, lng in zip(df['Neighborhood'], df['Latitude'], df['Longitude']):
    print(name)
        
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        500, 
        LIMIT)
        
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    # return only relevant information for each nearby venue
    venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    # return only relevant information for each nearby venue
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

nearby_venues.sample(10)


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
558,Thorncliffe Park,43.70142,-79.34905,Food Basics,43.705311,-79.347322,Supermarket
678,"Dufferin, Dovercourt Village",43.66505,-79.43891,Modern Furniture Knockoffs,43.668793,-79.438916,Furniture / Home Store
25,"Regent Park, Harbourfront",43.65512,-79.36264,Kibo Sushi House 希,43.658522,-79.359717,Sushi Restaurant
684,"Fairview, Henry Farm, Oriole",43.78097,-79.34781,TD Canada Trust,43.777763,-79.345974,Bank
201,"Garden District, Ryerson",43.65739,-79.37804,Marshalls,43.659308,-79.382462,Department Store
274,St. James Town,43.65215,-79.37587,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
1615,Davisville,43.70340,-79.38659,KFC,43.699300,-79.386900,Fast Food Restaurant
1968,Enclave of M5E,43.64869,-79.38544,Omg! Oh My Gyro!,43.650064,-79.391104,Souvlaki Shop
1478,Enclave of L4W,43.64869,-79.38544,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant
2238,Church and Wellesley,43.66659,-79.38133,The Drink,43.664167,-79.380149,Coffee Shop


In [14]:
df_crime = pd.read_csv(r'../data/crime_rate.csv'); df_crime.head()

,OBJECTID,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,...,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length
0,1,Yonge-St.Clair,97,12528,20,29,39,27,34,37,...,5,8,0,3,6,4.3,1.00,47.9,1.161315e+06,5873.270582
1,2,York University Heights,27,27593,271,296,361,344,357,370,...,46,37,39,38,28,36.3,-0.26,101.5,1.324666e+07,18504.777326
2,3,Lansing-Westgate,38,16164,44,80,68,85,75,72,...,5,5,11,6,11,7.0,0.83,68.1,5.346186e+06,11112.109625
3,4,Yorkdale-Glen Park,31,14804,106,136,174,161,175,209,...,14,26,23,20,29,22.5,0.45,195.9,6.038326e+06,10079.426920
4,5,Stonegate-Queensway,16,25051,88,71,76,95,87,82,...,8,4,6,7,4,6.0,-0.43,16.0,7.946202e+06,11853.189878


In [17]:
df_crime['Address'] = df_crime['Neighbourhood']+', Toronto, Ontario, Canada'

lat=[]
long=[]
for i, addr in enumerate(df_crime['Address']):
    with requests.Session() as session:
        g = geocoder.arcgis('{}'.format(addr),session=session)
    lat.append(g.latlng[0])
    long.append(g.latlng[1])

df_crime['Latitude'] = lat
df_crime['Longitude'] = long

,OBJECTID,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,...,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length,Address,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_crime.sample(10)

,OBJECTID,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,...,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length,Address,Latitude,Longitude
95,96,Willowdale East,51,50434,127,114,145,148,172,168,...,14,24,14.7,0.71,47.6,5.061016e+06,9482.383478,"Willowdale East, Toronto, Ontario, Canada",43.77249,-79.40041
101,102,Mount Pleasant West,104,29658,95,108,106,125,119,140,...,8,7,6.0,-0.13,23.6,1.343312e+06,5182.469731,"Mount Pleasant West, Toronto, Ontario, Canada",43.70541,-79.39256
59,60,Roncesvalles,86,14974,74,109,110,103,83,97,...,5,10,5.7,1.00,66.8,1.504527e+06,6163.829327,"Roncesvalles, Toronto, Ontario, Canada",43.64427,-79.44269
43,44,Kennedy Park,124,17123,216,248,218,195,215,224,...,20,6,7.3,-0.70,35.0,3.580725e+06,7606.631535,"Kennedy Park, Toronto, Ontario, Canada",43.72587,-79.26229
121,122,Long Branch,19,10084,60,67,60,67,57,59,...,3,2,2.3,-0.33,19.8,2.261880e+06,7806.777489,"Long Branch, Toronto, Ontario, Canada",43.59354,-79.53275
69,70,High Park North,88,22162,79,90,89,99,101,87,...,4,5,3.2,0.25,22.6,1.882284e+06,6319.774771,"High Park North, Toronto, Ontario, Canada",43.65769,-79.46449
79,80,Agincourt North,129,29113,67,76,77,71,79,79,...,4,2,4.7,-0.50,6.9,7.261857e+06,12381.264970,"Agincourt North, Toronto, Ontario, Canada",43.80930,-79.26707
42,43,Westminster-Branson,35,26274,99,102,131,104,114,116,...,4,2,4.7,-0.50,7.6,3.667762e+06,11495.365181,"Westminster-Branson, Toronto, Ontario, Canada",43.78206,-79.44936
122,123,Malvern,132,43794,215,280,280,289,272,333,...,16,10,9.0,-0.38,22.8,8.866244e+06,12081.208643,"Malvern, Toronto, Ontario, Canada",43.81023,-79.22038
55,56,Henry Farm,53,15723,64,69,72,74,81,85,...,5,14,6.0,1.80,89.0,2.593264e+06,8150.843708,"Henry Farm, Toronto, Ontario, Canada",43.77229,-79.34086
